In [1]:
%pwd

'c:\\Users\\91889\\OneDrive\\Desktop\\Diamond-Price-Prediction\\research'

In [2]:
import os

os.chdir("../")

%pwd

'c:\\Users\\91889\\OneDrive\\Desktop\\Diamond-Price-Prediction'

### Config Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    X_train_transformed_path: Path
    y_train_path: Path
    model_name: str
    n_estimators: int
    random_state: int

### Configuration Manager

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            X_train_transformed_path = config.X_train_transformed_path,
            y_train_path = config.y_train_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            random_state = params.random_state
        )

        return model_trainer_config

### Components - Model Trainer

In [6]:
import os
from mlProject import logger

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        X_train_transformed = pd.read_csv(self.config.X_train_transformed_path)
        y_train = pd.read_csv(self.config.y_train_path)

        model = RandomForestRegressor(n_estimators=self.config.n_estimators, random_state=self.config.random_state)

        # train the model
        model.fit(X_train_transformed, y_train.values.ravel())

        # save the model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

### Pipeline

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)

    model_trainer.train()

except Exception as e:
    raise e

[2024-10-20 18:08:58,535: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-20 18:08:58,535: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-20 18:08:58,545: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-20 18:08:58,545: INFO: common: created directory at: artifacts]
[2024-10-20 18:08:58,545: INFO: common: created directory at: artifacts/model_trainer]
